In [79]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model #draw graphs
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt #draw graphs
import os
from sklearn.model_selection import train_test_split # to create train, test, validation datasets
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for data preprocessing, image augmentation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3 #transfer learning model
from tensorflow.keras.utils import plot_model
import numpy as np
import pandas as pd

RestNet for classification: Frezzing all the layers and adding the last layer according to class


In [80]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model.layers:
  layer.trainable = False

In [81]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(5, activation = 'softmax')(x)

In [82]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [83]:
ROOT_DIR = '../input/cassava-leaf-disease-classification/'
TRAIN_DIR = ROOT_DIR + '/train_images/'
TEST_DIR = ROOT_DIR + '/test_images/'

Train, Test and Validation data creating

In [84]:
all_data_frame = pd.read_csv(ROOT_DIR + 'train.csv')
all_data_frame = all_data_frame.astype({"label": str})
other_data_frame, test_data_frame = train_test_split(
                        all_data_frame, 
                        test_size = 0.2, 
                        random_state = 2000)
print("test data size : "+str(test_data_frame.shape[0]))

In [85]:
train_data_frame, validation_data_frame = train_test_split(
                    other_data_frame, 
                    test_size = 0.2, 
                    random_state = 2000)

print("train data size : "+str(train_data_frame.shape[0]))
print("validation data size : "+str(validation_data_frame.shape[0]))

Data augmentation


In [86]:
train_image_data_generator = ImageDataGenerator(
                    rotation_range = 270, 
                    zoom_range = [0.05,1.0],
                    shear_range = 0.3, 
                    brightness_range = [0.5,1.5],
                    horizontal_flip = True,
                    vertical_flip = True,
                    width_shift_range = 0.4, 
                    height_shift_range = 0.4, 
                    fill_mode = 'nearest'
)

In [87]:
train_data_frame_iterator = train_image_data_generator.flow_from_dataframe(
                    train_data_frame,
                    directory = TRAIN_DIR,
                    x_col = "image_id",
                    y_col = "label",
                    target_size = (32,32),
                    class_mode = "sparse",
                    batch_size = 32,
                    shuffle = True,
                    seed = 2000,
                    interpolation = "nearest"
)

In [88]:
validation_data_frame_iterator =  ImageDataGenerator().flow_from_dataframe(
                    validation_data_frame,
                    directory = TRAIN_DIR,
                    x_col = "image_id",
                    y_col = "label",
                    target_size = (32,32),
                    class_mode = "sparse",
                    batch_size = 32,
                    shuffle = True,
                    seed = 2000,
                    interpolation = "nearest"
)

In [89]:
test_data_frame_iterator =  ImageDataGenerator().flow_from_dataframe(
                    test_data_frame,
                    directory = TRAIN_DIR,
                    x_col = "image_id",
                    y_col = "label",
                    target_size = (32,32),
                    class_mode = "sparse",
                    batch_size = 32,
                    shuffle = False,
                    seed = 2000,
                    interpolation = "nearest"
)

In [90]:
# Stop training when a monitored metric has stopped improving. usefull to prevent overfitting
early_stopping = EarlyStopping(monitor = 'val_loss',
                               patience = 4,
                               mode = 'min',
                               restore_best_weights = True)

# To save the Keras model in h5 format for future use.
model_checkpoint = ModelCheckpoint('model.h5', 
                             monitor = 'val_accuracy',
                             mode = 'max', 
                             save_best_only = True)

# When a measure no longer improves, slow down its learning rate.
reduce_learning_rate = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.3,
                              patience = 3,
                              mode = 'min')
TRAIN_STEPS_PER_EPOCH = train_data_frame_iterator.n//train_data_frame_iterator.batch_size
VALIDATION_STEPS_PER_EPOCH = validation_data_frame_iterator.n//validation_data_frame_iterator.batch_size
TEST_STEPS_PER_EPOCH = test_data_frame_iterator.n//test_data_frame_iterator.batch_size

In [91]:
head_model.summary()

In [92]:
plot_model(head_model, show_shapes = True)

In [93]:
history = head_model.fit(train_data_frame_iterator,
                    validation_data = validation_data_frame_iterator,
                    epochs = 20,
                    steps_per_epoch = TRAIN_STEPS_PER_EPOCH,
                    validation_steps = VALIDATION_STEPS_PER_EPOCH,
                    callbacks = [reduce_learning_rate,model_checkpoint,early_stopping]
                   )


In [94]:
validation_loss, validation_accuracy = head_model.evaluate(validation_data_frame_iterator, steps = VALIDATION_STEPS_PER_EPOCH)
print("validation accuracy: {:5.2f}%".format(100 * validation_accuracy))

In [95]:
test_loss, test_accuracy = head_model.evaluate(test_data_frame_iterator, steps = TEST_STEPS_PER_EPOCH)
print("test accuracy: {:5.2f}%".format(100 * test_accuracy))

In [96]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['accuracy'], 'b*-', label="train accuracy")
plt.plot(history.history['val_accuracy'], 'r*-', label="validation accuracy")
plt.grid()
plt.title("train accuracy vs validation accuracy RestNet152")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.savefig("restnet152_accuray.jpg")
plt.show()

In [97]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['loss'], 'b*-', label="train loss")
plt.plot(history.history['val_loss'], 'r*-', label="validation loss")
plt.grid()
plt.title("train loss vs validation loss for RestNet152")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.savefig("restnet152_loss.png")
plt.show()